<a href="https://colab.research.google.com/github/ethan060626/generative_ai/blob/main/week9_hw_ipynb%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 🎯 任務說明：將「員瑛式思考產生器」改寫為 Chain-of-Thought（CoT）推理過程

**原始任務：** 輸入一件小事或倒楣事，生成一段超級正向的社群貼文。

#### ✅ CoT 改寫版本流程：
1. **第一階段（思考階段）**：請 LLM 思考五種「為什麼這是一件超級幸運的事」的原因
2. **第二階段（產文階段）**：從這五個原因中挑出最有趣的一個，寫成社群發文（員瑛式風格 + 第一人稱 + emoji + "完全是 Lucky Vicky 呀!" 結尾）

這就是典型的 Planning 模式應用：先拆解、後執行。

#### 1. 讀入你的金鑰

請依你使用的服務, 決定讀入哪個金鑰

In [1]:
import os
from google.colab import userdata

In [2]:
#【使用 Groq】
api_key = userdata.get('Groq')
os.environ['GROQ_API_KEY']=api_key
provider = "groq"
model = "llama3-70b-8192"

In [3]:
!pip install aisuite[all]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 863.9/863.9 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.5/89.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 57.4 MB/s eta 0:00:00
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.16.1 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.


### 2. 使用 AISuite 的準備

In [4]:
import aisuite as ai

In [5]:
# provider_planner = "groq"
# model_planner="llama3-70b-8192"

# provider_writer = "groq"
# model_writer = "llama3-70b-8192"

provider_planner = "groq"
model_planner = "gemma2-9b-it"

provider_writer = "groq"
model_writer = "gemma2-9b-it"

#provider_reviewer = "openai"
#model_reviewer = "gpt-4o"

In [6]:
def reply(system="請用台灣習慣的中文回覆。",
          prompt="Hi",
          provider="groq",
          model="llama3-70b-8192"
          ):

    client = ai.Client()

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]


    response = client.chat.completions.create(model=f"{provider}:{model}", messages=messages)

    return response.choices[0].message.content

####  3. 打造二階段

In [7]:
system_planner = "請用台灣習慣的中文回應: 你是一個銷售員，你老闆要你每天交一份當天的銷售報告給他。 "
system_writer = "請用台灣習慣的中文回應: 你是一個老闆，你不管你的銷售員的報告多好，你都會有理由叫他回去重寫一份。"

In [8]:
def lucky_post(prompt):
    planning_prompt = f"使用者說：{prompt}"
    lucky_reasons = reply(system_planner, planning_prompt,
                          provider = provider_planner,
                          model = model_planner
                          )

    generation_prompt = f"這是我想到的五個理由：\n{lucky_reasons}\n\n請從中選出一個最好的理由，然後用這個理由把報告退回去。"
    final_post = reply(system_writer, generation_prompt,
                       provider = provider_writer,
                       model = model_writer
                       )

    return lucky_reasons, final_post

In [11]:
lucky_post("今天銷售很好。")

('太好了！ 今天客人反應都超好的，我們賣出了很多！請問您大概賣了多少呢？我需要把詳細資料寫在報告裡給老闆看。  😊 \n\n\n\n',
 '最好的理由應該是 **「請問您大概賣了多少呢？我需要把詳細資料寫在報告裡給老闆看。」**  \n\n因為這個理由顯得你很關心銷售數字，但同時也表明你希望能更有細節、更具數據佐證的分析。\n\n你可以回覆銷售員 seguinte:\n\n「**報告內容不錯，看起來今天銷售狀況很好！不過報告裡客人反應的部分好，客戶滿意度如何呢？再來，銷售數字是關鍵，請問您大概賣了多少呢？我希望可以在報告中用更具體的數字呈現，讓老闆看得更清楚。可以打一點詳細的銷售數據給我，我再幫你修改報告。**」\n\n\n\n這樣既給予讚賞，也能有效地要求銷售員提供更詳細的數據資料，提升報告的專業性和實用性。\n\n\n\n')

### 4. 用 Gradio 打造你的對話機器人 Web App!

In [9]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.3/323.3 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 110.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.0 MB/s eta 0:00:00


In [10]:
import gradio as gr

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("### 員工模擬器。")
    gr.Markdown("請輸入你今天的工作成果！")
    user_input = gr.Textbox(label="今天的成果…")
    btn = gr.Button("退回的理由")

    with gr.Row():
        out1 = gr.Textbox(label="🧠 五個理由（Planning CoT）")
        out2 = gr.Textbox(label="📣 最終理由")

    btn.click(lucky_post, inputs=[user_input], outputs=[out1, out2])

In [ ]:
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://2b3d3bb9318535b450.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
